In [2]:
%run ../Get-data.ipynb
%run ../GP-emulation.ipynb
%run ../Variable-selection.ipynb

Exception: File `'../GP-emulation.ipynb.py'` not found.

# Set-up

In [109]:
inputs = get_inputs()
my_inputs = list(set(get_inputs().columns)-set(['id']))

In [3]:
import pandas as pd
import numpy as np

def set_up(df):
    return df[list(set(my_inputs)-set(['id']))], pd.DataFrame(df["total"])

In [4]:
# Latitudes are 0.625 + k*1.25
my_lats = [0.625 + k*1.25 for k in range(-24, 7)]
# Longitudes are 0.9375 + k*1.875
my_lons = [0.9375 + k*1.875 for k in range(-24, 21)]

In [245]:
july = ["0" + str(int("0701") + k*int("0001")) for k in range(0, 31)]

In [246]:
hours = {"00":"01",
         "03":"04",
         "06":"07",
         "09":"10",
         "12":"13",
         "15":"16",
         "18":"19",
         "21":"22"}

# Set up data

In [247]:
def get_output_data(date, start_hour):
    outputs = get_outputs(start_lat=-30, end_lat=10,
                          start_lon=-45, end_lon=40,
                          start_day=date[2:4], start_month="07", start_year="2017", start_hour=start_hour,
                          end_day=date[2:4], end_month="07", end_year="2017", end_hour=hours[start_hour])

    my_data = get_data(inputs, outputs)
    my_data['longitude'] = [float(my_data.geoSurfaceTimePoint[row]['id'].split('_')[1]) for row in range(my_data.shape[0])]
    my_data['latitude'] = [float(my_data.geoSurfaceTimePoint[row]['id'].split('_')[0]) for row in range(my_data.shape[0])]
    
    return my_data

# New set up function

In [248]:
def get_map_data(df):
    
    # Set up the dataframe
    map_data = pd.DataFrame(columns=np.sort(my_inputs))

    # For each pixel, fill in a row
    for lat in my_lats: # in [-29.375, 9.375]:
        for lon in my_lons: # in [-44.0625, 38.4375]:
            
            my_df = df[(df.latitude == lat) & (df.longitude == round(lon,3))]

            # Fit emulator and get lengthscales
            X_train, y_train = set_up(my_df)
            my_model = train_GP_pipe(X_train, y_train)
            ard_results = ard(my_model, list(set(my_inputs)-set(['id'])))
            length_scales = pd.DataFrame(ard_results)['length_scale']
            
            # Append row
            new_row = pd.DataFrame(data=np.array(pd.DataFrame(np.append([lat,lon], np.array(length_scales))).transpose()),
                                   columns=np.append(['latitude', 'longitude',], np.sort(my_inputs)))
            map_data = pd.concat([map_data, new_row], axis=0)

    map_data["top_var"] = map_data.loc[:, my_inputs].idxmin(axis=1, skipna=True)
    return map_data
            

# Plot selected variable

In [249]:
import matplotlib as mpl

levels, categories = pd.factorize(my_inputs)
colors = [plt.cm.tab10(i) for i in levels]
handles = [mpl.patches.Patch(color=plt.cm.tab10(i), label=c) for i, c in enumerate(categories)]

In [252]:
def plot_my_top_variable(data_set, date, time, colors=colors, handles=handles, folder='variable-selection-july'):
    import matplotlib as mpl
    import matplotlib.pyplot as plt
    import cartopy.crs as ccrs

    """
    levels, categories = pd.factorize(data_set['top_var'])
    colors = [plt.cm.tab10(i) for i in levels]
    handles = [mpl.patches.Patch(color=plt.cm.tab10(i), label=c) for i, c in enumerate(categories)]
    """
    colors=[colors[idx] for idx in indices]
    handles=list(set([handles[idx] for idx in indices]))

    BBox = [np.min(data_set.longitude), np.max(data_set.longitude),
            np.min(data_set.latitude), np.max(data_set.latitude)]

    projection = ccrs.PlateCarree(central_longitude=0)
    fig = plt.figure(figsize=(20,20), facecolor='white')

    ax = fig.add_subplot(1, 1, 1, projection=projection)
    ax.coastlines()

    ax.set_extent(BBox, ccrs.PlateCarree())
    ax.gridlines(draw_labels=True, crs=projection)
    ax.set_facecolor("white")

    plt.scatter(data_set.longitude, data_set.latitude, c=colors)
    plt.legend(handles=handles, title='Smallest length scale')
    plt.xlabel('longitude')
    plt.ylabel('latitude')
    plt.savefig(folder + date + time)
    #plt.show()

In [ ]:
date = july[0]
hour = '00'
my_data = get_output_data(date, hour)
map_data = get_map_data(my_data)
plot_my_top_variable(map_data, date, hour, colors=colors, handles=handles, folder='variable-selection-july')

# Get maps

In [254]:
for date in july[0]:
    for hour in : # hours.keys():
        my_data = get_output_data(date, hour)
        map_data = get_map_data(my_data)
        plot_my_top_variable(map_data, date, hour, colors=colors, handles=handles, folder='variable-selection-july')

500 - NotClassified - c3.love.datetime.IsoDateTimeParser_errorInvalidStringAt [909.319]
message: "Invalid DateTime value '2017-07-T00:00:00.000' at 'T00:00:00.000' (expected ISO8601 format)."
JSON: {"spec": {"type": "FetchSpec", "filter": "latitude >= -30 && latitude <= 10 && longitude >= -45 && longitude <= 40 && time >= \"2017-07-T00:00:00.000\" && time <= \"2017-07-T01:00:00.000\"", "offset": 0, "limit": -1}}


C3RuntimeException: 500 - NotClassified - c3.love.datetime.IsoDateTimeParser_errorInvalidStringAt [909.319]
message: "Invalid DateTime value '2017-07-T00:00:00.000' at 'T00:00:00.000' (expected ISO8601 format)."
JSON: {"spec": {"type": "FetchSpec", "filter": "latitude >= -30 && latitude <= 10 && longitude >= -45 && longitude <= 40 && time >= \"2017-07-T00:00:00.000\" && time <= \"2017-07-T01:00:00.000\"", "offset": 0, "limit": -1}}